In [20]:
import os
import numpy as np
import pandas as pd
from lgbm import LGBM
from lm import LM
import math

from entities import *

from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [21]:
!rm -rf models
!mkdir models

!rm -rf output
!mkdir output

In [22]:
data_path = 'C:/Users/bill/Documents/projects/data/kaggle/house-prices/'
data = pd.read_csv(os.path.join(data_path, 'train.csv')).set_index(keys=['Id'])
test = pd.read_csv(os.path.join(data_path, 'test.csv')).set_index(keys=['Id'])

model_paths = { 'lv1': 'model/lv1/{}', 'lv2': 'model/lv2' }
output_paths = { 'lv1': 'output/lv1/{}', 'lv2': 'output/lv2' }

In [4]:
data.head(), test.head()

(    MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
 Id                                                                    
 1           60       RL         65.0     8450   Pave   NaN      Reg   
 2           20       RL         80.0     9600   Pave   NaN      Reg   
 3           60       RL         68.0    11250   Pave   NaN      IR1   
 4           70       RL         60.0     9550   Pave   NaN      IR1   
 5           60       RL         84.0    14260   Pave   NaN      IR1   
 
    LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
 Id                                                                     
 1          Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
 2          Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
 3          Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
 4          Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
 5          Lvl    AllPub       FR2       Gtl      NoRid

In [24]:
import featuretools as ft

# Make an entityset and add the entity
es = ft.EntitySet(id = 'prices')
es.entity_from_dataframe(entity_id = 'data', dataframe = data, 
                         make_index = True, index = 'index')

# Run deep feature synthesis with transformation primitives
feature_matrix, feature_defs = ft.dfs(
    entityset = es, target_entity = 'data',
    trans_primitives = ['add_numeric','multiply_numeric','mode']
)

feature_matrix.head()

MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
index                                                                    
0              60       RL         65.0     8450   Pave   NaN      Reg   
1              20       RL         80.0     9600   Pave   NaN      Reg   
2              60       RL         68.0    11250   Pave   NaN      IR1   
3              70       RL         60.0     9550   Pave   NaN      IR1   
4              60       RL         84.0    14260   Pave   NaN      IR1   

      LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
index                                                                     
0             Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
1             Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
2             Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
3             Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
4             Lvl    AllPub       FR2       Gtl      NoRidge       Norm   

      Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
index                                                                       
0           Norm     1Fam     2Story            7            5       2003   
1           Norm     1Fam     1Story            6            8       1976   
2           Norm     1Fam     2Story            7            5       2001   
3           Norm     1Fam     2Story            7            5       1915   
4           Norm     1Fam     2Story            8            5       2000   

       YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
index                                                                       
0              2003     Gable  CompShg     VinylSd     VinylSd    BrkFace   
1              1976     Gable  CompShg     MetalSd     MetalSd       None   
2              2002     Gable  CompShg     VinylSd     VinylSd    BrkFace   
3              1970     Gable  CompShg     Wd Sdng     Wd Shng       None   
4              2000     Gable  CompShg     VinylSd     VinylSd    BrkFace   

       MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond  \
index                                                                
0           196.0        Gd        TA      PConc       Gd       TA   
1             0.0        TA        TA     CBlock       Gd       TA   
2           162.0        Gd        TA      PConc       Gd       TA   
3             0.0        TA        TA     BrkTil       TA       Gd   
4           350.0        Gd        TA      PConc       Gd       TA   

      BsmtExposure BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  \
index                                                                  
0               No          GLQ         706          Unf           0   
1               Gd          ALQ         978          Unf           0   
2               Mn          GLQ         486          Unf           0   
3               No          ALQ         216          Unf           0   
4               Av          GLQ         655          Unf           0   

       BsmtUnfSF  TotalBsmtSF Heating HeatingQC CentralAir Electrical  \
index                                                                   
0            150          856    GasA        Ex          Y      SBrkr   
1            284         1262    GasA        Ex          Y      SBrkr   
2            434          920    GasA        Ex          Y      SBrkr   
3            540          756    GasA        Gd          Y      SBrkr   
4            490         1145    GasA        Ex          Y      SBrkr   

       1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  \
index                                                              
0           856       854             0       1710             1   
1          1262         0             0       1262             0   
2           920       866             0       1786             1   
3           961       756         

In [5]:
# check NA values
size = data.shape[0]
for c in list(data.columns):
    total = sum(data[c].isna())
    if total > 0:
        print(c)
        print('Miss\t{}'.format(total))
        print(data[c].value_counts())
        print('\n')

LotFrontage
Miss	259
60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
        ... 
106.0      1
38.0       1
138.0      1
140.0      1
137.0      1
Name: LotFrontage, Length: 110, dtype: int64


Alley
Miss	1369
Grvl    50
Pave    41
Name: Alley, dtype: int64


MasVnrType
Miss	8
None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64


MasVnrArea
Miss	8
0.0      861
72.0       8
180.0      8
108.0      8
120.0      7
        ... 
651.0      1
337.0      1
415.0      1
293.0      1
621.0      1
Name: MasVnrArea, Length: 327, dtype: int64


BsmtQual
Miss	37
TA    649
Gd    618
Ex    121
Fa     35
Name: BsmtQual, dtype: int64


BsmtCond
Miss	37
TA    1311
Gd      65
Fa      45
Po       2
Name: BsmtCond, dtype: int64


BsmtExposure
Miss	38
No    953
Av    221
Gd    134
Mn    114
Name: BsmtExposure, dtype: int64


BsmtFinType1
Miss	37
Unf    430
GLQ    418
ALQ    220
BLQ    148
Rec    133
LwQ     74
Name: BsmtFinType1, dtype: int64


BsmtFin

In [6]:
# check NA values
size = test.shape[0]
for c in list(test.columns):
    total = sum(test[c].isna())
    if total > 0:
        print(c)
        print('Miss\t{}'.format(total))
        print(test[c].value_counts())
        print('\n')

MSZoning
Miss	4
RL         1114
RM          242
FV           74
C (all)      15
RH           10
Name: MSZoning, dtype: int64


LotFrontage
Miss	227
60.0     133
80.0      68
70.0      63
50.0      60
75.0      52
        ... 
22.0       1
136.0      1
149.0      1
31.0       1
131.0      1
Name: LotFrontage, Length: 115, dtype: int64


Alley
Miss	1352
Grvl    70
Pave    37
Name: Alley, dtype: int64


Utilities
Miss	2
AllPub    1457
Name: Utilities, dtype: int64


Exterior1st
Miss	1
VinylSd    510
MetalSd    230
HdBoard    220
Wd Sdng    205
Plywood    113
CemntBd     65
BrkFace     37
WdShing     30
AsbShng     24
Stucco      18
BrkComm      4
CBlock       1
AsphShn      1
Name: Exterior1st, dtype: int64


Exterior2nd
Miss	1
VinylSd    510
MetalSd    233
HdBoard    199
Wd Sdng    194
Plywood    128
CmentBd     66
Wd Shng     43
BrkFace     22
Stucco      21
AsbShng     18
Brk Cmn     15
ImStucc      5
CBlock       2
Stone        1
AsphShn      1
Name: Exterior2nd, dtype: int64


MasVnr

In [7]:
# replace NA values
default_values = {
    'LotFrontage': 0.0,
    'Alley': 'NA',
    'MasVnrType': 'None',
    'MasVnrArea': 0.0,
    'BsmtQual': 'NA',
    'BsmtCond': 'NA',
    'BsmtExposure': 'NA',
    'BsmtFinType1': 'NA',
    'BsmtFinType2': 'NA',
    'Electrical': 'NA',
    'FireplaceQu': 'NA',
    'GarageType': 'NA',
    'GarageFinish': 'NA',
    'GarageQual': 'NA',
    'GarageCond': 'NA',
    'PoolQC': 'NA',
    'Fence': 'NA',
    'MiscFeature': 'NA',
    
    'MSZoning': 'RL',
    'Utilities': 'AllPub',
    'Exterior1st': 'VinylSd',
    'Exterior2nd': 'VinylSd',
    'BsmtFinSF1': 0.0,
    'BsmtFinSF2': 0.0,
    'BsmtUnfSF': 0.0,
    'TotalBsmtSF': 0.0,
    'BsmtFullBath': 0.0,
    'BsmtHalfBath': 0.0,
    'Functional': 'Typ',
    'GarageCars': 0.0,
    'GarageArea': 0.0,
    'SaleType': 'WD',
    'GarageYrBlt': 0
}

def cleanse(df):
    for c, v in default_values.items():
        df.loc[df[c].isna(), c] = v
    #df.loc[df['GarageYrBlt'].isna(), 'GarageYrBlt'] = df['YearBuilt']
    df.loc[df['KitchenQual'].isna(), 'KitchenQual'] = df['OverallQual']
    return df

data = cleanse(data)
test = cleanse(test)

In [8]:
de = DummyEncoder()
data = de.run(data, remove_last=False)
test = de.run(test, remove_last=False)

A dummy encoder has been constructed.


In [9]:
# apply log transformation on y
y = 'SalePrice'
data[y] = data[y].apply(lambda x : math.log(x))
data.shape, test.shape

((1460, 304), (1459, 303))

In [10]:
K = 10
X = [ c for c in data.columns if c != y ]
# remove linearly dependent columns
#_, r = np.linalg.qr(np.matrix(data[X]))
#print('before {}'.format(len(X)))
#X = np.array(X)[np.where(np.abs(np.diagonal(r))>=0.1)[0]].tolist()
#print('after {}'.format(len(X)))
cvf = CvFold(K, data, X, y, stratified=False, seed=0, shuffle=True)
folds = cvf.get_nested_folds()

In [11]:
rmses = []
for k in range(len(folds)):
    lgbm = LGBM(model_paths['lv1'].format(k), output_paths['lv1'].format(k))
    rmses.append(lgbm.train(y, folds[k]['lv1']))

rmse = round(np.mean(rmses), 4)
print('avg rmse: {}'.format(rmse))

avg rmse: 0.1209


In [12]:
rmses = []

for k in range(len(folds)):
    en = LM(model_paths['lv1'].format(k), output_paths['lv1'].format(k), model='en')
    rmses.append(en.train(y, folds[k]['lv1'], l1_ratio=0.5))

rmse = round(np.mean(rmses), 4)
print('avg rmse: {}'.format(rmse))

avg rmse: 0.1372


In [13]:
lv2folds = []
for k in range(len(folds)):
    path = output_paths['lv1'].format(k)
    files = [ path + '/' + f for f in os.listdir(path) ]
    
    # retrieve level 2 validation
    lv2_valid = folds[k]['lv2']['valid']
    
    train, valid = None, None
    for i, m in enumerate(['lgbm', 'en']):
        
        # read predictions for each level 1 fold and each method
        tmp = pd.concat([ 
            pd.read_csv(f, index_col=0) for f in files if m + '_' in f 
        ]).sort_index()
        
        if i == 0:
            train = tmp[['y']].rename(columns={
                'y': y
            })
        train[m] = tmp['yhat']
        
        # apply model to predict on level 2 dataset
        model = None
        if m == 'lgbm':
            model = LGBM(model_paths['lv1'].format(k), output_paths['lv1'].format(k))
        if m == 'en':
            model = LM(model_paths['lv1'].format(k), output_paths['lv1'].format(k), model='en')
            
        if i == 0:
            valid = lv2_valid[[y]].copy()
        valid[m] = model.predict(lv2_valid.drop(columns=[y]))
    
    lv2folds.append({
        'train': train,
        'valid': valid
    })

    
model_paths = { 'lv1': 'model/lv1/{}', 'lv2': 'model/lv2' }
output_paths = { 'lv1': 'output/lv1/{}', 'lv2': 'output/lv2' }

In [14]:
lm = LM(model_paths['lv2'], output_paths['lv2'], model='lm')
lm_rmse = lm.train(y, lv2folds)
lm_rmse

0.1243

In [15]:
lv2_test = pd.DataFrame(index=test.index)

for i, m in enumerate(['lgbm', 'en']):
    lv2_test[m] = 0.0
    for k in range(K):
        model = None
        if m == 'lgbm':
            model = LGBM(model_paths['lv1'].format(k), output_paths['lv1'].format(k))
        if m == 'en':
            model = LM(model_paths['lv1'].format(k), output_paths['lv1'].format(k), model='en')
        lv2_test[m] += model.predict(test[X]) / K
    
lm = LM(model_paths['lv2'], output_paths['lv2'], model='lm')
lv2_test[y] = np.exp(lm.predict(lv2_test))

score_path = os.path.join(data_path, 'scores.csv')
lv2_test[[y]].to_csv(score_path)
name = 'lm_blend_{}'.format(lm_rmse)
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f $score_path -m $name

Successfully submitted to House Prices: Advanced Regression Techniques



  0%|          | 0.00/35.1k [00:00<?, ?B/s]
100%|##########| 35.1k/35.1k [00:00<00:00, 62.1kB/s]


In [16]:
lv2_test[y] = np.exp(lv2_test['lgbm'] / 2 + lv2_test['en'] / 2)
lv2_test[[y]].to_csv(score_path)
name = 'lm_avg_{}'.format(lm_rmse)
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f $score_path -m $name

Successfully submitted to House Prices: Advanced Regression Techniques


  0%|          | 0.00/35.1k [00:00<?, ?B/s]
100%|##########| 35.1k/35.1k [00:00<00:00, 73.1kB/s]


In [19]:
# LGBM only
lv2_test = pd.DataFrame(index=test.index)
lv2_test[y] = 0.0
for k in range(K):
    model = LGBM(model_paths['lv1'].format(k), output_paths['lv1'].format(k))
    lv2_test[y] += np.exp(model.predict(test[X])) / K
    
score_path = os.path.join(data_path, 'scores.csv')
lv2_test[[y]].to_csv(score_path)

name = 'lgbm_{}'.format(lm_rmse)
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f $score_path -m $name

403 - Your team has used its submission allowance (10 of 10). This resets at midnight UTC (20 hours from now).



  0%|          | 0.00/35.1k [00:00<?, ?B/s]
100%|##########| 35.1k/35.1k [00:00<00:00, 55.9kB/s]


In [18]:
# LM only
lv2_test = pd.DataFrame(index=test.index)
lv2_test[y] = 0.0
for k in range(K):
    model = LM(model_paths['lv1'].format(k), output_paths['lv1'].format(k), 'en')
    lv2_test[y] += np.exp(model.predict(test[X])) / K
    
score_path = os.path.join(data_path, 'scores.csv')
lv2_test[[y]].to_csv(score_path)

name = 'en_{}'.format(lm_rmse)
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f $score_path -m $name

403 - Your team has used its submission allowance (10 of 10). This resets at midnight UTC (20 hours from now).



  0%|          | 0.00/35.1k [00:00<?, ?B/s]
100%|##########| 35.1k/35.1k [00:00<00:00, 86.2kB/s]
